In [1]:
import keras
import scipy
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import layers

import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from numba import jit, float64, int64

plt.rc('font', size=12)          # controls default text sizes
plt.rc('axes', titlesize=12)     # fontsize of the axes title
plt.rc('axes', labelsize=15)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=12)    # fontsize of the tick labels
plt.rc('ytick', labelsize=12)    # fontsize of the tick labels
plt.rc('legend', fontsize=15)    # legend fontsize

In [2]:
def make_train_test(data, test_size=0.3):
    sbd = data[data['sbj'] == 0]
    X_train, X_test, y1, y2 = train_test_split(sbd, sbd, test_size=test_size, random_state=1111)
    for i in range(1, data.sbj.max()+1):
        sbd = data[data['sbj'] == i]
        tr, ts, y1, y2 = train_test_split(sbd, sbd, test_size=test_size, random_state=1111)
        X_train = pd.concat([X_train, tr])
        X_test = pd.concat([X_test, ts])
        
    return X_train.reset_index(drop=True), X_test.reset_index(drop=True)

In [3]:
@jit(nopython=True)
def Race(v, b, ndt=0, dt=0.001):
    x = np.array([0.0, 0.0])
    rt = 0
    ch = -1
    stop = False
    while not stop:
        x[0] += v[0]*dt + np.random.normal(0, 1)*np.sqrt(dt)
        x[1] += v[1]*dt + np.random.normal(0, 1)*np.sqrt(dt)
        rt += dt
        if x[0]>= b and x[1]<b:
            ch = 0.0
            stop = True
        elif x[1]>= b and x[0]<b:
            ch = 1.0
            stop = True
        elif x[0]>= b and x[1]>=b:
            rt = 0
            x = np.array([0.0, 0.0])
    return rt+ndt, ch

In [4]:
data_df = pd.read_csv('../../_00_data/Kr2010_prep.csv', index_col=0)

min_rt = data_df.groupby(by='sbj')['rt'].min()

In [5]:
tr_df, ts_df = make_train_test(data_df, test_size=0.4)

obs_train = np.asarray(tr_df[['rt', 'Right', 'sbj']])
X_train = np.asarray(tr_df[['item_value_1', 'item_value_2', 'gaze_1', 'gaze_2']])

obs_test = np.asarray(ts_df[['rt', 'Right', 'sbj']])
X_test = np.asarray(ts_df[['item_value_1', 'item_value_2', 'gaze_1', 'gaze_2']])

print(X_train.shape, X_test.shape)

(2096, 4) (1426, 4)


In [6]:
def fpt(t, b, v):
    return tf.cast(tf.math.greater(t, 0), tf.float32)*(b/(2*np.pi*t**3)**(1/2) * keras.backend.exp(-1/(2*t) * (v*t-b)**2)) + tf.cast(tf.math.less_equal(t, 0), tf.float32)*0.1**10

def phi(x):
    return 0.5*tf.math.erf(x/tf.sqrt(2.0)) + 0.5

def cdf(t, b, v):
    return tf.cast(tf.math.greater(t, 0), tf.float32)*(phi((v*t-b)/tf.sqrt(t)) + tf.exp(2*v*b)*phi((-v*t-b)/tf.sqrt(t))) + (tf.cast(tf.math.less_equal(t, 0), tf.float32)*0)

def G2(data, pred):

    sbj_indices = tf.cast(data[:, 2], tf.int32)

    # Gather values directly using TensorFlow operations
    gathered_min_rt = tf.gather(min_rt, sbj_indices)
    sigmoid_weights = tf.gather(model.layers[-1].weights[-1], sbj_indices)
    ndt = gathered_min_rt * tf.math.sigmoid(sigmoid_weights)

    thr_weights = tf.gather(model.layers[-1].weights[-2], sbj_indices)
    thr = tf.math.log(1 + tf.exp(thr_weights))

    scl_weights = tf.gather(model.layers[-1].weights[-3], sbj_indices)
    scl = tf.math.log(1 + tf.exp(scl_weights))

    ll1 = (1 - data[:, 1]) * fpt(data[:, 0]-ndt, thr, scl*pred[:, 0]) * (1.0 - cdf(data[:, 0]-ndt, thr, scl*pred[:, 1]))
    ll2 = (data[:, 1]) * fpt(data[:, 0]-ndt, thr, scl*pred[:, 1]) * (1.0 - cdf(data[:, 0]-ndt, thr, scl*pred[:, 0]))

    return -2*tf.reduce_sum(tf.math.log(tf.maximum(ll1 + ll2, 1e-14)), axis=-1)

In [7]:
class Out_layer(keras.layers.Layer):
    def __init__(self, units=2, input_dim=32, n_sbj_prms=10):
        super().__init__()
        self.w = self.add_weight(shape=(input_dim, units), initializer="random_normal", trainable=True)
        self.b = self.add_weight(shape=(units,), initializer="uniform", trainable=True)
        
        self.scaling = self.add_weight(shape=(n_sbj_prms,), initializer="uniform", trainable=True)
        self.threshold = self.add_weight(shape=(n_sbj_prms,), initializer="uniform", trainable=True)
        self.non_decision_time = self.add_weight(shape=(n_sbj_prms,), initializer="uniform", trainable=True)

    def call(self, inputs):
        return tf.math.log(1 + tf.exp(tf.matmul(inputs, self.w) + self.b))
    
def make_model(layers_neuron, input_shape, output_shape, activation, n_sbj_prms):
    keras.backend.clear_session()

    model = keras.Sequential()
    model.add(layers.Input(shape=(input_shape,)))
    
    for l in layers_neuron:
        model.add(layers.Dense(l, activation=activation))
    
    model.add(Out_layer(units=output_shape, input_dim=layers_neuron[-1], n_sbj_prms=n_sbj_prms))

    print(model.summary())
    
    return model

In [8]:
layers_neuron = [10, 10, 8, 5]
input_shape = 4
output_shape = 2
activation = 'tanh'
n_sbj = tr_df.sbj.nunique()

model = make_model(layers_neuron, input_shape, output_shape, activation, n_sbj)

model.build()
model.compile(optimizer="RMSprop", loss=G2)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                50        
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 8)                 88        
                                                                 
 dense_3 (Dense)             (None, 5)                 45        
                                                                 
 out_layer (Out_layer)       (None, 2)                 129       
                                                                 
Total params: 422 (1.65 KB)
Trainable params: 422 (1.65 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [9]:
sbj_indices = tf.cast(obs_train[:, 2], tf.int32)
obs_train = tf.cast(obs_train, tf.float32)
X_train = tf.convert_to_tensor(X_train)
X_train = tf.cast(X_train, tf.float32)

min_rt = tf.convert_to_tensor(min_rt)
min_rt = tf.cast(min_rt, tf.float32)

In [10]:
history = model.fit(X_train, 
                    obs_train, 
                    shuffle=False, 
                    batch_size=X_train.shape[0], 
                    epochs=4000)

Epoch 1/4000
1/1 [==============================] - 1s 570ms/step - loss: 12781.5107
Epoch 2/4000
1/1 [==============================] - 0s 2ms/step - loss: 12714.3770
Epoch 3/4000
1/1 [==============================] - 0s 2ms/step - loss: 12666.5801
Epoch 4/4000
1/1 [==============================] - 0s 2ms/step - loss: 12627.0352
Epoch 5/4000
1/1 [==============================] - 0s 2ms/step - loss: 12592.2090
Epoch 6/4000
1/1 [==============================] - 0s 2ms/step - loss: 12560.4219
Epoch 7/4000
1/1 [==============================] - 0s 2ms/step - loss: 12530.7363
Epoch 8/4000
1/1 [==============================] - 0s 2ms/step - loss: 12502.5928
Epoch 9/4000
1/1 [==============================] - 0s 2ms/step - loss: 12475.6289
Epoch 10/4000
1/1 [==============================] - 0s 3ms/step - loss: 12449.6191
Epoch 11/4000
1/1 [==============================] - 0s 2ms/step - loss: 12424.4092
Epoch 12/4000
1/1 [==============================] - 0s 2ms/step - loss: 12399.9004

1/1 [==============================] - 0s 2ms/step - loss: 11103.0000
Epoch 99/4000
1/1 [==============================] - 0s 2ms/step - loss: 11091.3848
Epoch 100/4000
1/1 [==============================] - 0s 3ms/step - loss: 11079.8281
Epoch 101/4000
1/1 [==============================] - 0s 2ms/step - loss: 11068.3262
Epoch 102/4000
1/1 [==============================] - 0s 2ms/step - loss: 11056.8828
Epoch 103/4000
1/1 [==============================] - 0s 2ms/step - loss: 11045.4961
Epoch 104/4000
1/1 [==============================] - 0s 2ms/step - loss: 11034.1660
Epoch 105/4000
1/1 [==============================] - 0s 2ms/step - loss: 11022.8926
Epoch 106/4000
1/1 [==============================] - 0s 3ms/step - loss: 11011.6748
Epoch 107/4000
1/1 [==============================] - 0s 2ms/step - loss: 11000.5137
Epoch 108/4000
1/1 [==============================] - 0s 2ms/step - loss: 10989.4102
Epoch 109/4000
1/1 [==============================] - 0s 2ms/step - loss: 10978.3

1/1 [==============================] - 0s 3ms/step - loss: 10211.9121
Epoch 195/4000
1/1 [==============================] - 0s 3ms/step - loss: 10204.5459
Epoch 196/4000
1/1 [==============================] - 0s 4ms/step - loss: 10197.2109
Epoch 197/4000
1/1 [==============================] - 0s 3ms/step - loss: 10189.9062
Epoch 198/4000
1/1 [==============================] - 0s 3ms/step - loss: 10182.6299
Epoch 199/4000
1/1 [==============================] - 0s 3ms/step - loss: 10175.3867
Epoch 200/4000
1/1 [==============================] - 0s 3ms/step - loss: 10168.1699
Epoch 201/4000
1/1 [==============================] - 0s 3ms/step - loss: 10160.9824
Epoch 202/4000
1/1 [==============================] - 0s 3ms/step - loss: 10153.8242
Epoch 203/4000
1/1 [==============================] - 0s 3ms/step - loss: 10146.6953
Epoch 204/4000
1/1 [==============================] - 0s 3ms/step - loss: 10139.5957
Epoch 205/4000
1/1 [==============================] - 0s 4ms/step - loss: 10132.

1/1 [==============================] - 0s 2ms/step - loss: 9587.1113
Epoch 292/4000
1/1 [==============================] - 0s 2ms/step - loss: 9580.7227
Epoch 293/4000
1/1 [==============================] - 0s 3ms/step - loss: 9574.3164
Epoch 294/4000
1/1 [==============================] - 0s 2ms/step - loss: 9567.8916
Epoch 295/4000
1/1 [==============================] - 0s 2ms/step - loss: 9561.4473
Epoch 296/4000
1/1 [==============================] - 0s 2ms/step - loss: 9554.9775
Epoch 297/4000
1/1 [==============================] - 0s 2ms/step - loss: 9548.4863
Epoch 298/4000
1/1 [==============================] - 0s 2ms/step - loss: 9541.9678
Epoch 299/4000
1/1 [==============================] - 0s 3ms/step - loss: 9535.4229
Epoch 300/4000
1/1 [==============================] - 0s 2ms/step - loss: 9528.8496
Epoch 301/4000
1/1 [==============================] - 0s 2ms/step - loss: 9522.2500
Epoch 302/4000
1/1 [==============================] - 0s 2ms/step - loss: 9515.6230
Epoch 3

1/1 [==============================] - 0s 2ms/step - loss: 8928.2402
Epoch 389/4000
1/1 [==============================] - 0s 3ms/step - loss: 8921.5898
Epoch 390/4000
1/1 [==============================] - 0s 2ms/step - loss: 8914.9502
Epoch 391/4000
1/1 [==============================] - 0s 2ms/step - loss: 8908.3203
Epoch 392/4000
1/1 [==============================] - 0s 2ms/step - loss: 8901.7041
Epoch 393/4000
1/1 [==============================] - 0s 2ms/step - loss: 8895.0957
Epoch 394/4000
1/1 [==============================] - 0s 2ms/step - loss: 8888.5020
Epoch 395/4000
1/1 [==============================] - 0s 3ms/step - loss: 8881.9160
Epoch 396/4000
1/1 [==============================] - 0s 2ms/step - loss: 8875.3438
Epoch 397/4000
1/1 [==============================] - 0s 2ms/step - loss: 8868.7812
Epoch 398/4000
1/1 [==============================] - 0s 2ms/step - loss: 8862.2305
Epoch 399/4000
1/1 [==============================] - 0s 2ms/step - loss: 8855.6904
Epoch 4

...

1/1 [==============================] - 0s 2ms/step - loss: 5435.3691
Epoch 3417/4000
1/1 [==============================] - 0s 2ms/step - loss: 5434.3335
Epoch 3418/4000
1/1 [==============================] - 0s 3ms/step - loss: 5434.0742
Epoch 3419/4000
1/1 [==============================] - 0s 2ms/step - loss: 5434.0396
Epoch 3420/4000
1/1 [==============================] - 0s 2ms/step - loss: 5434.0034
Epoch 3421/4000
1/1 [==============================] - 0s 2ms/step - loss: 5434.3994
Epoch 3422/4000
1/1 [==============================] - 0s 2ms/step - loss: 5433.7563
Epoch 3423/4000
1/1 [==============================] - 0s 2ms/step - loss: 5434.3701
Epoch 3424/4000
1/1 [==============================] - 0s 3ms/step - loss: 5435.2520
Epoch 3425/4000
1/1 [==============================] - 0s 2ms/step - loss: 5436.3066
Epoch 3426/4000
1/1 [==============================] - 0s 2ms/step - loss: 5437.5527
Epoch 3427/4000
1/1 [==============================] - 0s 2ms/step - loss: 5441.5

1/1 [==============================] - 0s 3ms/step - loss: 5429.8350
Epoch 3513/4000
1/1 [==============================] - 0s 3ms/step - loss: 5431.8579
Epoch 3514/4000
1/1 [==============================] - 0s 3ms/step - loss: 5431.4160
Epoch 3515/4000
1/1 [==============================] - 0s 3ms/step - loss: 5434.5664
Epoch 3516/4000
1/1 [==============================] - 0s 3ms/step - loss: 5436.6426
Epoch 3517/4000
1/1 [==============================] - 0s 3ms/step - loss: 5436.2988
Epoch 3518/4000
1/1 [==============================] - 0s 2ms/step - loss: 5433.0405
Epoch 3519/4000
1/1 [==============================] - 0s 2ms/step - loss: 5432.5098
Epoch 3520/4000
1/1 [==============================] - 0s 2ms/step - loss: 5431.4854
Epoch 3521/4000
1/1 [==============================] - 0s 2ms/step - loss: 5429.7295
Epoch 3522/4000
1/1 [==============================] - 0s 3ms/step - loss: 5428.6743
Epoch 3523/4000
1/1 [==============================] - 0s 2ms/step - loss: 5429.5

1/1 [==============================] - 0s 2ms/step - loss: 5426.7686
Epoch 3609/4000
1/1 [==============================] - 0s 2ms/step - loss: 5425.1279
Epoch 3610/4000
1/1 [==============================] - 0s 2ms/step - loss: 5426.1807
Epoch 3611/4000
1/1 [==============================] - 0s 2ms/step - loss: 5425.8760
Epoch 3612/4000
1/1 [==============================] - 0s 3ms/step - loss: 5426.4268
Epoch 3613/4000
1/1 [==============================] - 0s 2ms/step - loss: 5426.1890
Epoch 3614/4000
1/1 [==============================] - 0s 2ms/step - loss: 5427.0430
Epoch 3615/4000
1/1 [==============================] - 0s 2ms/step - loss: 5428.1279
Epoch 3616/4000
1/1 [==============================] - 0s 2ms/step - loss: 5429.6904
Epoch 3617/4000
1/1 [==============================] - 0s 2ms/step - loss: 5427.5762
Epoch 3618/4000
1/1 [==============================] - 0s 3ms/step - loss: 5427.6084
Epoch 3619/4000
1/1 [==============================] - 0s 2ms/step - loss: 5425.4

1/1 [==============================] - 0s 2ms/step - loss: 5421.2695
Epoch 3705/4000
1/1 [==============================] - 0s 2ms/step - loss: 5420.7236
Epoch 3706/4000
1/1 [==============================] - 0s 2ms/step - loss: 5420.4141
Epoch 3707/4000
1/1 [==============================] - 0s 2ms/step - loss: 5420.0518
Epoch 3708/4000
1/1 [==============================] - 0s 2ms/step - loss: 5420.6812
Epoch 3709/4000
1/1 [==============================] - 0s 2ms/step - loss: 5420.4629
Epoch 3710/4000
1/1 [==============================] - 0s 2ms/step - loss: 5421.4780
Epoch 3711/4000
1/1 [==============================] - 0s 3ms/step - loss: 5421.3574
Epoch 3712/4000
1/1 [==============================] - 0s 2ms/step - loss: 5423.5938
Epoch 3713/4000
1/1 [==============================] - 0s 2ms/step - loss: 5423.5845
Epoch 3714/4000
1/1 [==============================] - 0s 2ms/step - loss: 5425.4668
Epoch 3715/4000
1/1 [==============================] - 0s 2ms/step - loss: 5422.7

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [11]:
pred_v = model.predict(X_test)
threshold = np.asanyarray(tf.math.log(1 + tf.exp(model.layers[-1].weights[-2])))
ndt = np.asanyarray(np.asanyarray(min_rt)*tf.math.sigmoid(model.layers[-1].weights[-1]))

45/45 [==============================] - 0s 283us/step


In [12]:
pred_bhv = {'sbj': [],
            'rt': [],
            'ch': [],
            'value_diff': [],
            'gaze_diff': []}

scaling = np.asanyarray(tf.math.log(1 + tf.exp(model.layers[-1].weights[-3])))
scaling = np.asanyarray([scaling[i] for i in ts_df['sbj']])

for n in range(20):
    for s in range(ts_df.shape[0]):
        rt, ch = Race(scaling[s]*pred_v[s,:], threshold[ts_df.sbj[s]], ndt[ts_df.sbj[s]])
        pred_bhv['sbj'].append(ts_df.sbj[s])
        pred_bhv['rt'].append(rt)
        pred_bhv['ch'].append(ch)
        pred_bhv['value_diff'].append(ts_df.item_value_2[s] - ts_df.item_value_1[s])
        pred_bhv['gaze_diff'].append(ts_df['Dwell Time Advantage (R - L)'][s])
        
pred_bhv = pd.DataFrame(pred_bhv)

In [13]:
pred_bhv.to_csv('Kr2010_out_of_sample.csv')
ts_df.to_csv('Kr2010_test.csv')